In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from WindPy import w
import pyodbc
import datetime as dt
%matplotlib inline

In [2]:
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [3]:
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.0.185.138;DATABASE=JYDB;UID=js_dev;PWD=js_dev')

In [4]:
report_date = '2020-03-31'
last_date = dt.datetime.today() - dt.timedelta(1)

In [5]:
def get_wind_code(c):
    if c.startswith('0') or c.startswith('3'):
        return c + '.SZ'
    else:
        return c + '.SH'

# 基金池中基金重仓股

In [9]:
funds = pd.read_excel('../data/基金池.xlsx')

In [11]:
funds.head()

,基金代码,基金名称,基金规模
0,660010.OF,农银汇理策略精选,2.024779e+09
1,519773.OF,交银数据产业,3.680433e+09
2,720001.OF,财通价值动量,2.352315e+09
3,002692.OF,富国创新科技,7.496134e+09
4,000404.OF,易方达新兴成长,6.474656e+09


In [12]:
set_codes = '(' + ','.join(['\'%s\''%(x.rstrip('.OF')) for x in funds['基金代码']]) + ')'

In [13]:
query = """
    select ksp.ReportDate, t.SecuCode, t.SecuAbbr, t2.SecuCode, t2.SecuAbbr, ksp.SharesHolding, ksp.ChangeOfSharesHolding, ksp.RatioInNV
    from MF_KeyStockPortfolio as ksp, SecuMain as t, SecuMain as t2
    where ksp.ReportDate = '%s'
        and ksp.InnerCode = t.InnerCode
        and ksp.StockInnerCode = t2.InnerCode
        and t.SecuCode in %s
"""%(report_date, set_codes)

In [14]:
df = pd.read_sql_query(query, con=conn)

In [15]:
df.columns = ['ReportDate', 'SecuCode', 'SecuAbbr', 'StockCode', 'StockAbbr', 'SharesHolding', 'ChangeOfSharesHolding', 'RatioInNV']

In [16]:
df.head()

,ReportDate,SecuCode,SecuAbbr,StockCode,StockAbbr,SharesHolding,ChangeOfSharesHolding,RatioInNV
0,2020-03-31,000251,工银地产,600036,招商银行,13599900.0,-1400152.0,0.0977
1,2020-03-31,000251,工银地产,002142,宁波银行,16297177.0,-1591705.0,0.0837
2,2020-03-31,000251,工银地产,601318,中国平安,5197030.0,197030.0,0.0800
3,2020-03-31,000251,工银地产,600030,中信证券,13964500.0,-1368800.0,0.0689
4,2020-03-31,000251,工银地产,000001,平安银行,15012900.0,12900.0,0.0428


In [17]:
stocks = df.groupby(['StockCode', 'StockAbbr']).sum()[['SharesHolding', 'ChangeOfSharesHolding']]

In [18]:
stocks['RatioInNV'] = df.groupby(['StockCode', 'StockAbbr']).mean()['RatioInNV']

In [19]:
stocks['WindCode'] = [get_wind_code(x) for x in stocks.index.levels[0]]

In [20]:
data = w.wss(stocks['WindCode'].tolist(), 'close', "tradeDate=%s"%(last_date.strftime('%Y-%m-%d')))

In [21]:
stocks['Close'] = data.Data[0]
stocks['MarketValue'] = stocks['SharesHolding'] * stocks['Close']

In [22]:
stocks['ChangeOfSharesHoldingPct'] = stocks['ChangeOfSharesHolding'] / (stocks['SharesHolding'] - stocks['ChangeOfSharesHolding'])

In [23]:
stocks = stocks.sort_values('RatioInNV', ascending=False)

In [24]:
stocks.head()

,,SharesHolding,ChangeOfSharesHolding,RatioInNV,WindCode,Close,MarketValue,ChangeOfSharesHoldingPct
StockCode,StockAbbr,,,,,,,
601318,中国平安,39675662.0,297006.0,0.10415,601318.SH,71.01,2.817369e+09,0.007542
000513,丽珠集团,7935421.0,-23400.0,0.10050,000513.SZ,49.10,3.896292e+08,-0.002940
603866,桃李面包,3027766.0,-307760.0,0.09970,603866.SH,49.32,1.493294e+08,-0.092267
300699,光威复材,2215925.0,1446645.0,0.09580,300699.SZ,62.70,1.389385e+08,1.880518
600885,宏发股份,10488860.0,5559991.0,0.09450,600885.SH,38.55,4.043456e+08,1.128046


### 行业分类

In [26]:
industry = pd.read_excel('../data/申万行业分类表.xlsx')

In [27]:
industry.head()

,金融地产,可选消费,必选医药,信息科技,其他经济敏感,其他经济不敏感
0,银行Ⅱ(申万),汽车整车(申万),食品加工(申万),半导体(申万),煤炭开采Ⅱ(申万),农产品加工(申万)
1,多元金融Ⅱ(申万),汽车零部件Ⅱ(申万),化学制药(申万),其他电子Ⅱ(申万),金属非金属新材料(申万),饲料Ⅱ(申万)
2,券商Ⅱ(申万),白色家电(申万),中药Ⅱ(申万),元件Ⅱ(申万),其他采掘Ⅱ(申万),渔业(申万)
3,保险Ⅱ(申万),景点(申万),生物制品Ⅱ(申万),光学光电子(申万),采掘服务Ⅱ(申万),种植业(申万)
4,房地产开发Ⅱ(申万),酒店Ⅱ(申万),医疗器械Ⅱ(申万),电子制造Ⅱ(申万),工业金属(申万),畜禽养殖Ⅱ(申万)


In [28]:
industry2theme = {}
for col in industry.columns:
    for i in industry[col]:
        if type(i) == str:
            industry2theme[i.rstrip('(申万)')] = col
            industry2theme[i.rstrip('Ⅱ(申万)')] = col
#             print(i.rstrip('Ⅱ(申万)'), col)
industry2theme['证券Ⅱ'] = '金融地产'

In [29]:
data = w.wss(stocks['WindCode'].tolist(), 'industry_sw', 'industryType=2')

In [30]:
stocks['industry_sw'] = data.Data[0]

In [31]:
stocks['industry_theme'] = [industry2theme[x] for x in stocks['industry_sw']]

In [32]:
stocks.head()

,,SharesHolding,ChangeOfSharesHolding,RatioInNV,WindCode,Close,MarketValue,ChangeOfSharesHoldingPct,industry_sw,industry_theme
StockCode,StockAbbr,,,,,,,,,
601318,中国平安,39675662.0,297006.0,0.10415,601318.SH,71.01,2.817369e+09,0.007542,保险Ⅱ,金融地产
000513,丽珠集团,7935421.0,-23400.0,0.10050,000513.SZ,49.10,3.896292e+08,-0.002940,化学制药,必选医药
603866,桃李面包,3027766.0,-307760.0,0.09970,603866.SH,49.32,1.493294e+08,-0.092267,食品加工,必选医药
300699,光威复材,2215925.0,1446645.0,0.09580,300699.SZ,62.70,1.389385e+08,1.880518,化学纤维,其他经济敏感
600885,宏发股份,10488860.0,5559991.0,0.09450,600885.SH,38.55,4.043456e+08,1.128046,高低压设备,信息科技


In [34]:
stocks.to_excel('../data/基金池基金重仓股.xlsx')

# 全部基金重仓股

In [35]:
query = """
    select ksp.ReportDate, t.SecuCode, t.SecuAbbr, t2.SecuCode, t2.SecuAbbr, ksp.SharesHolding, ksp.ChangeOfSharesHolding, ksp.RatioInNV
    from MF_KeyStockPortfolio as ksp, SecuMain as t, SecuMain as t2
    where ksp.ReportDate = '%s'
        and ksp.InnerCode = t.InnerCode
        and ksp.StockInnerCode = t2.InnerCode
"""%(report_date)

In [36]:
df = pd.read_sql_query(query, con=conn)

In [37]:
df.columns = ['ReportDate', 'SecuCode', 'SecuAbbr', 'StockCode', 'StockAbbr', 'SharesHolding', 'ChangeOfSharesHolding', 'RatioInNV']

In [38]:
stocks = df.groupby(['StockCode', 'StockAbbr']).sum()[['SharesHolding', 'ChangeOfSharesHolding']]

In [39]:
stocks['RatioInNV'] = df.groupby(['StockCode', 'StockAbbr']).mean()['RatioInNV']

In [40]:
stocks['WindCode'] = [get_wind_code(x) for x in stocks.index.levels[0]]

In [41]:
data = w.wss(stocks['WindCode'].tolist(), 'close', "tradeDate=%s"%(last_date.strftime('%Y-%m-%d')))

In [42]:
stocks['Close'] = data.Data[0]
stocks['MarketValue'] = stocks['SharesHolding'] * stocks['Close']

In [43]:
stocks = stocks.sort_values('MarketValue', ascending=False)

In [44]:
stocks['ChangeOfSharesHoldingPct'] = stocks['ChangeOfSharesHolding'] / (stocks['SharesHolding'] - stocks['ChangeOfSharesHolding'])

In [45]:
stocks.to_excel('../data/全部基金基金重仓股.xlsx')